<a href="https://colab.research.google.com/github/Nithil3007/InceptionResnet-V1-and-ElasticNetAttack/blob/main/Elastic%20net%20attack%20without%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 27.7 MB/s 


In [ ]:
pip install git+https://github.com/CNOCycle/cleverhans.git@feature/tf2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/CNOCycle/cleverhans.git (to revision feature/tf2.x) to /tmp/pip-req-build-nnhz_51j
  Running command git clone -q https://github.com/CNOCycle/cleverhans.git /tmp/pip-req-build-nnhz_51j
  Running command git checkout -b feature/tf2.x --track origin/feature/tf2.x
  Switched to a new branch 'feature/tf2.x'
  Branch 'feature/tf2.x' set up to track remote branch 'feature/tf2.x' from 'origin'.
     |████████████████████████████████| 154 kB 24.7 MB/s 
     |████████████████████████████████| 41 kB 479 kB/s 
  Created wheel for cleverhans: filename=cleverhans-3.0.1-py3-none-any.whl size=254394 sha256=2225ca8250bdf3357baf5000c36b6a84215b55b5bf4b719e5657087de854fe7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-nfexi0q0/wheels/7b/f7/6b/d754a65b70ab265387116e0a3b9c7bdb7adedc4e7afd04815f
Successfully built cleverhans


In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cleverhans

plt.ion()  

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INF = float("inf")

In [ ]:
def elastic_net(
    model_fn,
    x,
    n_classes,
    y=None,
    targeted=False,
    lr=5e-3,
    confidence=0,
    clip_min=0,
    clip_max=1,
    initial_const=1e-2,
    binary_search_steps=5,
    max_iterations=10,
):

    def compare(pred, label, is_logits=False):
        # Convert logits to predicted class if necessary
        if is_logits:
            pred_copy = pred.clone().detach()
            pred_copy[label] += -confidence if targeted else confidence
            pred = torch.argmax(pred_copy)

        return pred == label if targeted else pred != label

    if y is None:
        # Using model predictions as ground truth to avoid label leaking
        pred = model_fn(x)
        y = torch.argmax(pred, 1)

    # Initialize some values needed for binary search on const
    lower_bound = [0.0] * len(x)
    upper_bound = [1e10] * len(x)
    const = x.new_ones(len(x), 1) * initial_const

    o_bestl2 = [INF] * len(x)
    o_bestscore = [-1.0] * len(x)
    x = torch.clamp(x, clip_min, clip_max)
    ox = x.clone().detach()  # save the original x
    o_bestattack = x.clone().detach()

    # Map images into the tanh-space
    x = (x - clip_min) / (clip_max - clip_min)
    x = torch.clamp(x, 0, 1)
    x = x * 2 - 1
    x = torch.arctanh(x * 0.999999)

    # Prepare some variables
    modifier = torch.zeros_like(x, requires_grad=True)
    y_onehot = torch.nn.functional.one_hot(y, n_classes).to(torch.float)

    # Define loss functions and optimizer
    f_fn = lambda real, other, targeted: torch.max(
        ((other - real) if targeted else (real - other)) + confidence,
        torch.tensor(0.0).to(real.device),
    )
    l2dist_fn = lambda x, y: torch.pow(x - y, 2).sum(list(range(len(x.size())))[1:])
    optimizer = torch.optim.Adam([modifier], lr=lr)

    # Outer loop performing binary search on const
    for outer_step in range(binary_search_steps):
        # Initialize some values needed for the inner loop
        bestl2 = [INF] * len(x)
        bestscore = [-1.0] * len(x)

        # Inner loop performing attack iterations
        for i in range(max_iterations):
            # One attack step
            new_x = (torch.tanh(modifier + x) + 1) / 2
            new_x = new_x * (clip_max - clip_min) + clip_min
            logits = model_fn(new_x)

            real = torch.sum(y_onehot * logits, 1)
            other, _ = torch.max((1 - y_onehot) * logits - y_onehot * 1e4, 1)

            optimizer.zero_grad()
            f = f_fn(real, other, targeted)
            l2 = l2dist_fn(new_x, ox)
            loss = (const * f + l2).sum()
            loss.backward()
            optimizer.step()

            # Update best results
            for n, (l2_n, logits_n, new_x_n) in enumerate(zip(l2, logits, new_x)):
                y_n = y[n]
                succeeded = compare(logits_n, y_n, is_logits=True)
                if l2_n < o_bestl2[n] and succeeded:
                    pred_n = torch.argmax(logits_n)
                    o_bestl2[n] = l2_n
                    o_bestscore[n] = pred_n
                    o_bestattack[n] = new_x_n
                    # l2_n < o_bestl2[n] implies l2_n < bestl2[n] so we modify inner loop variables too
                    bestl2[n] = l2_n
                    bestscore[n] = pred_n
                elif l2_n < bestl2[n] and succeeded:
                    bestl2[n] = l2_n
                    bestscore[n] = torch.argmax(logits_n)

        # Binary search step
        for n in range(len(x)):
            y_n = y[n]

            if compare(bestscore[n], y_n) and bestscore[n] != -1:
                # Success, divide const by two
                upper_bound[n] = min(upper_bound[n], const[n])
                if upper_bound[n] < 1e9:
                    const[n] = (lower_bound[n] + upper_bound[n]) / 2
            else:
                # Failure, either multiply by 10 if no solution found yet
                # or do binary search with the known upper bound
                lower_bound[n] = max(lower_bound[n], const[n])
                if upper_bound[n] < 1e9:
                    const[n] = (lower_bound[n] + upper_bound[n]) / 2
                else:
                    const[n] *= 10

    return o_bestattack.detach()

In [ ]:
path='/content/drive/MyDrive/College/intern/TASK3/RETINAL DATASET'
dir_list = os.listdir(path)
print(dir_list)

['TEST3', 'TRAIN2', 'TRAIN', 'TEST', 'TEST2', 'Test_temp', 'Train_temp', 'InceptionResnetV1_TensorFlow_model', 'VALIDATION', 'InceptonResnetV1_Pytorch', 'model.txt']


In [ ]:
base_dir = os.path.join("/content/drive/MyDrive/College/intern/TASK3/RETINAL DATASET/")
print('Base directory --> ', os.listdir(base_dir))

Base directory -->  ['TEST3', 'TRAIN2', 'TRAIN', 'TEST', 'TEST2', 'Test_temp', 'Train_temp', 'InceptionResnetV1_TensorFlow_model', 'VALIDATION', 'InceptonResnetV1_Pytorch', 'model.txt']


In [ ]:
train_dir = os.path.join(base_dir + "TRAIN/")
print("Train Directory --> ", os.listdir(train_dir))

validation_dir = os.path.join(base_dir + "VALIDATION/")
print("Test Directory --> ", os.listdir(validation_dir))

test_dir = os.path.join(base_dir + "TEST2/")
print("Test2 Directory --> ", os.listdir(test_dir))

Train Directory -->  ['CNV', 'NORMAL', 'train_images.csv', 'train_images.gsheet']
Test Directory -->  ['NORMAL', 'CNV']
Test2 Directory -->  ['CNV_TEST_2', 'NORMAL_TEST_2']


In [ ]:
data_dir = base_dir
TRAIN = 'TRAIN'
VAL = 'VALIDATION'
TEST = 'TEST2'

# The pretrained model takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=2
    )
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)

Loaded 15000 images under TRAIN
Loaded 1000 images under VALIDATION
Loaded 4000 images under TEST2
Classes: 
['CNV', 'NORMAL']


In [ ]:
from facenet_pytorch import InceptionResnetV1
model = InceptionResnetV1(pretrained='vggface2',classify=True,num_classes=2).eval()
model=model.cuda()

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
#UNFREEZES TRAINING FOR THE FIRST 9 LAYERS OF THE MODEL, FREEZES TRINING FOR THE REST
for param in model.parameters():
          param.requires_grad = True
#print(newmodel.children)
ct = 0
for child in model.children():
  ct += 1
  if ct < 9:
     for param in child.parameters():
          param.requires_grad = False

In [ ]:
#summary(model,(3, 224, 224))

In [ ]:
if use_gpu:
    model.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [ ]:
def advTrain(torch_model, train_loader, test_loader,
        nb_epochs=2, batch_size=8, train_end=-1, test_end=-1, learning_rate=0.0001, optimizer=None):

    train_loss = []
    total = 0
    correct = 0
    step = 0
    train_batches = len(train_loader)
    test_batches = len(test_loader)
    for epoch in range(nb_epochs):
        print("Epoch {}/{}".format(epoch, nb_epochs))
        print('-' * 10)
        
        loss_train = 0
        acc_train = 0
        loss_train_adv = 0
        acc_train_adv = 0
        
        loss_test = 0
        acc_test = 0
        loss_test_adv = 0
        acc_test_adv = 0

        model.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data)
            torch.cuda.empty_cache()

        model.train(False)
        model.eval()

        for i, data in enumerate(dataloaders[TEST]):
              if i % 100 == 0:
                  print("\rTesting batch {}/{}".format(i, test_batches / 2), end='', flush=True)
                
              # Use half training dataset
              if i >= test_batches / 2:
                  break

              inputs, labels = data
              inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
              inputs = elastic_net(torch_model,inputs,n_classes=2,y=labels,clip_min=0.,clip_max=1.)
              optimizer.zero_grad()
            
              outputs = model(inputs)
            
              _, preds = torch.max(outputs.data, 1)
              loss = criterion(outputs, labels)
            
              loss.backward()
              optimizer.step()
            
              loss_test_adv += loss.data
              acc_test_adv += torch.sum(preds == labels.data)
              torch.cuda.empty_cache()

        avg_acc_train = acc_train * 2 / dataset_sizes[TRAIN]
        print("\nEpoch {} result: ".format(epoch))
        print()
        print("Training acc : {:.4f}".format(avg_acc_train))
        print()

        avg_acc_test_adv = acc_test_adv * 2 / dataset_sizes[TEST]
        print("\nEpoch {} result: ".format(epoch))
        print()
        print("Adversarial Testing acc : {:.4f}".format(avg_acc_test_adv))
        print('-' * 10)

In [ ]:
advtrainedmodel = advTrain(model, dataloaders[TRAIN], dataloaders[VAL],
        nb_epochs=2, batch_size=8, train_end=-1, test_end=-1, learning_rate=0.0001,optimizer=optimizer_ft)

Epoch 0/2
----------
Testing batch 0/62.5
Epoch 0 result: 

Training acc : 0.7113


Epoch 0 result: 

Adversarial Testing acc : 0.0755
----------
Epoch 1/2
----------
Testing batch 0/62.5
Epoch 1 result: 

Training acc : 0.8249


Epoch 1 result: 

Adversarial Testing acc : 0.0705
----------
